In [11]:
import os
import pickle
import numpy as np
import cv2
import tensorflow as tf
import random
from keras_facenet import FaceNet
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.image import resize
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomBrightness, RandomContrast, RandomTranslation, Input

In [12]:
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH,640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT,480)

faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
faceRecognition = pickle.load(open("model_v1_facenet.pickle", "rb"))

while(True):
    ret, frame = cap.read()
    frame = cv2.flip(frame, 1)
    faces = faceCascade.detectMultiScale(
        frame,     
        scaleFactor=1.2,
        minNeighbors=5,     
        minSize=(20, 20)
    )
    for (x,y,w,h) in faces:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),2)
        roi_color = frame[y:y+h, x:x+w]
        roi_color = cv2.flip(roi_color, 1)

        label = faceRecognition.predict(roi_color)
        label = label[:12] + "..." if len(label) > 10 else label
        
        cv2.putText(frame, label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,0,255), 2)
    cv2.imshow('video',frame)
    
    k = cv2.waitKey(30) & 0xff
    if k == 27: # press 'ESC' to quit
        break
    os.system('cls')
cap.release()
cv2.destroyAllWindows()

In [13]:
class FaceNetModel():
    def load_data(self, data_path: str):
        train_image_list = []
        test_image_list = []
        train_class_list = []
        test_class_list = []
        
        self.input_size = (160, 160)
        faceCascade = cv2.CascadeClassifier('Cascades/haarcascade_frontalface_default.xml')
        
        for label in os.listdir(data_path):
            if(len(os.listdir(os.path.join(data_path, label))) < 5):
                continue
            test = 0
            for j, filename in enumerate(os.listdir(os.path.join(data_path, label))):
                if j >= 10:
                    break
                filename = os.path.join(data_path, label, filename)
                image = load_img(filename)
                image = np.array(image)
                faces = faceCascade.detectMultiScale(
                    image,     
                    scaleFactor=1.2,
                    minNeighbors=5,     
                    minSize=(20, 20)
                )
                for (x, y, w, h) in faces:
                    if w == 0 or h == 0:
                        continue
                    image_roi = image[y:y+h, x:x+w]
                    image_roi = img_to_array(image_roi)
                    image_roi = resize(image_roi, self.input_size)
                    if test == 0:
                        test_image_list.append(image_roi)
                        test_class_list.append(label)
                        test = 1
                    else:
                        train_image_list.append(image_roi)
                        train_class_list.append(label)
                    print(f"Index {j}. {filename}")
        os.system("cls")
        encoder = LabelEncoder()
        test_class_list = encoder.fit_transform(test_class_list)
        train_class_list = encoder.transform(train_class_list)
        self.label_names = encoder.classes_
        
        self.train_data = (np.array(train_image_list), np.array(train_class_list))
        self.test_data = (np.array(test_image_list), np.array(test_class_list))
        print(f"Train dataset len: {len(train_image_list)}")
        print(f"Test dataset len: {len(test_image_list)}")

    def augmentation(self, images, labels, max_val = 3):
        augmented_images = []
        augmented_labels = []
        data_augmentation = Sequential([
            Input(shape=images[0].shape),
            RandomBrightness(factor=0.2),
            RandomContrast(factor=0.2),
            RandomFlip("horizontal"),
            RandomTranslation(height_factor=0.1, width_factor=0.1),
            RandomRotation(factor=0.1)
        ])
        for image, label in zip(images, labels):
            for _ in range(max_val):
                augmented_image = data_augmentation(tf.expand_dims(image, 0))
                augmented_image = resize(tf.squeeze(augmented_image), self.input_size)
                augmented_images.append(augmented_image)
                augmented_labels.append(label)
                
        return np.array(augmented_images), augmented_labels

    def save(self, save_path):
        self.X_train = []
        self.y_train = []
        self.X_val = []
        self.y_val = []
        self.X_test = []
        self.y_test = []
        
        model_save_path = save_path + "_model.h5"
        self.model.save(model_save_path)

        # Save the rest of the class attributes
        with open(save_path + "_attributes.pickle", "wb") as file:
            pickle.dump({
                "label_names": self.label_names,
                "input_size": self.input_size,
            }, file)
            
    @classmethod
    def load(cls, save_path):
        with open(save_path + "_attributes.pickle", "rb") as file:
            attributes = pickle.load(file)

        instance = cls()
        instance.label_names = attributes["label_names"]
        instance.input_size = attributes["input_size"]

        # Load the Keras model
        model_save_path = save_path + "_model.h5"
        instance.model = tf.keras.models.load_model(model_save_path)

        print(f"Model and attributes loaded from {save_path}")
        return instance
    
    def normalize(self, image):
        image = image.astype('float32')
        mean = image.mean()
        std = image.std()
        return (image - mean) / std
    
    def fit(self, augmentation=True):        
        X_train, y_train = self.train_data
        X_test, y_test = self.test_data
        
        if augmentation:
            X_train, y_train = self.augmentation(X_train, y_train, 3)
            
        print(X_train.shape)
        embedder = FaceNet()
        X_train_embeddings = embedder.embeddings(X_train)
        X_test_embeddings = embedder.embeddings(X_test)
        
        new_X_train_embeddings = []
        new_Y_train = []
        embeddings_dict = {}

        for embed, label in zip(X_train_embeddings, y_train):
            if label not in embeddings_dict:
                embeddings_dict[label] = {'count': 0, 'sum': np.zeros_like(embed)}
            embeddings_dict[label]['count'] += 1
            embeddings_dict[label]['sum'] += embed

        for label in embeddings_dict:
            avg_embed = embeddings_dict[label]["sum"] / embeddings_dict[label]["count"]
            new_X_train_embeddings.append(avg_embed)
            new_Y_train.append(label)
        
        y_pred = []
        for x1 in X_test_embeddings:
            best_similarity = -1
            best_match = -1
            for i, x0 in enumerate(new_X_train_embeddings):
                pred = cosine_similarity([x0], [x1])[0][0]
                if pred > best_similarity:
                    best_similarity = pred
                    best_match = new_Y_train[i]
            y_pred.append(best_match)
        
        self.cr = classification_report(y_test, y_pred)
        self.cm = confusion_matrix(y_test, y_pred)         
        self.train_embedding = new_X_train_embeddings
        self.train_label = new_Y_train

    def predict(self, predict_image):
        embedder = FaceNet()
        predict_image = img_to_array(predict_image)
        predict_image = resize(predict_image, self.input_size)
        predict_image = np.expand_dims(predict_image, axis=0)
        predict_image_embeddings = embedder.embeddings(predict_image)[0]
        
        best_similarity = -1
        best_match = -1
        for i, x_train in enumerate(self.train_embedding):
            pred = cosine_similarity([x_train], [predict_image_embeddings])[0][0]
            if pred > best_similarity:
                best_similarity = pred
                best_match = self.train_label[i]
        return self.label_names[best_match]


In [14]:
model = FaceNetModel()

In [15]:
model.load_data("Dataset")

Index 0. Dataset\Abdullah_Gul\Abdullah_Gul_0001.jpg
Index 1. Dataset\Abdullah_Gul\Abdullah_Gul_0002.jpg
Index 2. Dataset\Abdullah_Gul\Abdullah_Gul_0003.jpg
Index 3. Dataset\Abdullah_Gul\Abdullah_Gul_0004.jpg
Index 4. Dataset\Abdullah_Gul\Abdullah_Gul_0005.jpg
Index 5. Dataset\Abdullah_Gul\Abdullah_Gul_0008.jpg
Index 6. Dataset\Abdullah_Gul\Abdullah_Gul_0009.jpg
Index 7. Dataset\Abdullah_Gul\Abdullah_Gul_0010.jpg
Index 8. Dataset\Abdullah_Gul\Abdullah_Gul_0011.jpg
Index 9. Dataset\Abdullah_Gul\Abdullah_Gul_0012.jpg
Index 0. Dataset\Adrien_Brody\Adrien_Brody_0001.jpg
Index 1. Dataset\Adrien_Brody\Adrien_Brody_0002.jpg
Index 2. Dataset\Adrien_Brody\Adrien_Brody_0003.jpg
Index 3. Dataset\Adrien_Brody\Adrien_Brody_0004.jpg
Index 4. Dataset\Adrien_Brody\Adrien_Brody_0005.jpg
Index 5. Dataset\Adrien_Brody\Adrien_Brody_0006.jpg
Index 6. Dataset\Adrien_Brody\Adrien_Brody_0007.jpg
Index 7. Dataset\Adrien_Brody\Adrien_Brody_0008.jpg
Index 8. Dataset\Adrien_Brody\Adrien_Brody_0009.jpg
Index 9. Dat

In [16]:
print(len(model.label_names))
print(model.label_names)

425
['Abdullah_Gul' 'Adrien_Brody' 'Ai_Sugiyama' 'Al_Gore' 'Al_Sharpton'
 'Alan_Greenspan' 'Alastair_Campbell' 'Albert_Costa' 'Alejandro_Toledo'
 'Ali_Naimi' 'Allyson_Felix' 'Alvaro_Uribe' 'Alvian Devano Sugiarto'
 'Amelia_Vega' 'Amelie_Mauresmo' 'Ana_Guevara' 'Ana_Palacio'
 'Andre_Agassi' 'Andy_Roddick' 'Angela_Bassett' 'Angela_Merkel'
 'Angelina_Jolie' 'Ann_Veneman' 'Anna_Kournikova' 'Antonio_Banderas'
 'Antonio_Palocci' 'Ari_Fleischer' 'Ariel_Sharon' 'Arminio_Fraga'
 'Arnold_Schwarzenegger' 'Arnoldo_Aleman' 'Ashanti' 'Atal_Bihari_Vajpayee'
 'Ben_Affleck' 'Benazir_Bhutto' 'Benjamin_Netanyahu' 'Bernard_Law'
 'Bertie_Ahern' 'Bill_Clinton' 'Bill_Frist' 'Bill_Gates' 'Bill_Graham'
 'Bill_McBride' 'Bill_Simon' 'Billy_Crystal' 'Binyamin_Ben-Eliezer'
 'Bob_Graham' 'Bob_Hope' 'Bob_Stoops' 'Boris_Becker' 'Brad_Johnson'
 'Britney_Spears' 'Bulent_Ecevit' 'Calista_Flockhart' 'Cameron_Diaz'
 'Carla_Del_Ponte' 'Carlos_Menem' 'Carlos_Moya' 'Carmen_Electra'
 'Carrie-Anne_Moss' 'Catherine_Deneuve' 'Ca

In [17]:
model.fit(augmentation=True)


(8442, 160, 160, 3)

14/14 [==============================] - 8s 559ms/step


d:\Coding\Anaconda\envs\computer_vision\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Coding\Anaconda\envs\computer_vision\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Coding\Anaconda\envs\computer_vision\lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [18]:
print(model.cr)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      1.00      1.00         1
           2       1.00      1.00      1.00         1
           3       1.00      1.00      1.00         1
           4       1.00      1.00      1.00         1
           5       1.00      1.00      1.00         1
           6       1.00      1.00      1.00         1
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         1
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         1
          12       1.00      1.00      1.00         1
          13       1.00      1.00      1.00         1
          14       1.00      1.00      1.00         1
          15       1.00      1.00      1.00         1
          16       1.00      1.00      1.00         1
          17       1.00    

In [19]:
print(model.cm)

[[1 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]
 [0 0 0 ... 0 0 1]]


In [20]:
model.save("model_v1_facenet")